In [1]:
import requests 
import pandas as pd 
import json
from datetime import datetime
import time
from internal_data import weather_api
import os

year = '2025'
month = '01'

# Камс\сибирская
# lat = '56.48'
# lon = '84.98'

# Черных
lat = '56.51'
lon = '85.03'

# year = input('Введите год: ')
# month = input('Введите месяц (цифрой): ')

# if len(month) == 1:
#     month = '0' + month

# lat = input('Введите широту в формате ХХ.ХХ: ')
# lon = input('Введите долготу в формате ХХ.ХХ: ')

# Проверем кратность на 400
if int(year) % 400 == 0:
    if_leap = 1

# Проверим кратность на 4 и отсутсвтие кратности на 100
elif int(year) % 4 == 0 and int(year) % 100 != 0:
    if_leap = 1

# Остальные случае не попадают под свойства високосности года, выведем результат
else:
    if_leap = 0


if month in ['01', '03', '05', '07', '08', '10', '12']:
    limit_day = 32

elif month in ['02'] and if_leap == 0:
    limit_day = 29
    
elif month in ['02'] and if_leap == 1:
    limit_day = 30

else:
    limit_day = 31


col_list = ['dt', 'sunrise', 'sunset', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg', 'main', 'description']

month_df = pd.DataFrame(columns=col_list)

for day in range(1, limit_day):
        
    for hour in range(0,24):
        
        if day in list(range(10)):
            day = f'0{day}'
        
        if hour in list(range(10)):
            hour = f'0{hour}'

        date_to_stamp = f'{day}/{month}/{year}/{hour}'
        
        time_stamp = int(time.mktime(datetime.strptime(date_to_stamp, "%d/%m/%Y/%H").timetuple()))

        
        url_arch = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&units={'metric'}&dt={time_stamp}&appid={weather_api}'

        response_arch = requests.get(url_arch)

        data_arch = response_arch.json()
        
        df_day_hour_arch = pd.json_normalize(data_arch['data'][0])
        
        df_day_hour_arch['main'] = df_day_hour_arch['weather'][0][0]['main']
        df_day_hour_arch['description'] = df_day_hour_arch['weather'][0][0]['description']
        df_day_hour_arch = df_day_hour_arch.drop('weather', axis=1)
        
        month_df = pd.concat([month_df, df_day_hour_arch])
        
        time.sleep(3)
        
        
# Сохранение с шатмпом времени

month_df_csv = month_df.copy()

saving_path = f'{month}-{year}'

if not os.path.exists(saving_path):
    os.makedirs(saving_path)

exist_files = os.listdir(f'{saving_path}')

datestamp_file_name = f'{saving_path}_datestamps.csv'

n = 1

while datestamp_file_name in exist_files:
    print(f'in while, in {n} - iteration')
    datestamp_file_name_list = datestamp_file_name.split('.')
    
    if n == 1:
        datestamp_file_name = str(f'{datestamp_file_name_list[0]}_{n}.csv')
    else:
        orig_file_name = datestamp_file_name_list[0][:-2]
        datestamp_file_name = str(f'{orig_file_name}_{n}.csv')
            
    n += 1

month_df_csv.to_csv(f'{saving_path}/{datestamp_file_name}', index=False)

excel_save_name_ts = f'{saving_path}/{datestamp_file_name[:-3]}xlsx'

with pd.ExcelWriter(excel_save_name_ts, engine='xlsxwriter') as writer:  

    month_df_csv.to_excel(
        writer, sheet_name=f'{month}-{year}_hour_weather', index=False
        )
    
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(0, 2, 20)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(11, 11, 11)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(12, 12, 20)


# Сохранение с читаемой датой 

feats_to_convert = ['dt', 'sunrise', 'sunset']

month_df_readable_date = month_df.copy()

def ts_convert(ts):
    return datetime.fromtimestamp(ts).strftime('%d-%m-%Y %H:%M:%S')

for feat in feats_to_convert:
    month_df_readable_date[feat] = month_df_readable_date[feat].apply(ts_convert)
    
    
readable_date_file_name = f'{saving_path}_readable_date.csv'

exist_files = os.listdir(f'{saving_path}')

n = 1

while readable_date_file_name in exist_files:
    print(f'in while, in {n} - iteration')
    readable_date_file_name_list = readable_date_file_name.split('.')
    
    if n == 1:
        readable_date_file_name = str(f'{readable_date_file_name_list[0]}_{n}.csv')
    else:
        orig_file_name = readable_date_file_name_list[0][:-2]
        readable_date_file_name = str(f'{orig_file_name}_{n}.csv')
            
    n += 1

month_df_readable_date.to_csv(f'{saving_path}/{readable_date_file_name}', index=False)

excel_save_name_readable_date = f'{saving_path}/{readable_date_file_name[:-3]}xlsx'

with pd.ExcelWriter(excel_save_name_readable_date, engine='xlsxwriter') as writer:  

    month_df_readable_date.to_excel(
        writer, sheet_name=f'{month}-{year}_hour_weather', index=False
        )
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(0, 2, 20)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(11, 11, 11)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(12, 12, 20)
   


KeyError: 'data'

In [2]:
month_df.shape

(338, 17)

In [80]:
import os

month_df_csv = month_df.copy()

saving_path = f'{month}-{year}'

if not os.path.exists(saving_path):
    os.makedirs(saving_path)

exist_files = os.listdir(f'{saving_path}')

datestamp_file_name = f'{saving_path}_datestamps.csv'

n = 1

while datestamp_file_name in exist_files:
    print(f'in while, in {n} - iteration')
    datestamp_file_name_list = datestamp_file_name.split('.')
    
    if n == 1:
        datestamp_file_name = str(f'{datestamp_file_name_list[0]}_{n}.csv')
    else:
        orig_file_name = datestamp_file_name_list[0][:-2]
        datestamp_file_name = str(f'{orig_file_name}_{n}.csv')
            
    n += 1

month_df_csv.to_csv(f'{saving_path}/{datestamp_file_name}', index=False)

excel_save_name_ts = f'{saving_path}/{datestamp_file_name[:-3]}xlsx'

with pd.ExcelWriter(excel_save_name_ts, engine='xlsxwriter') as writer:  

    month_df_csv.to_excel(
        writer, sheet_name=f'{month}-{year}_hour_weather', index=False
        )


In [59]:
month_df_csv

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,main,description,visibility,snow.1h,wind_gust,rain.1h
0,1735664400,1735700070,1735725140,-12.50,-19.50,1030,89,-13.78,100,3.87,208,Clouds,overcast clouds,NaN,NaN,NaN,NaN
0,1735668000,1735700070,1735725140,-12.26,-19.26,1030,92,-13.18,100,4.00,200,Snow,light snow,10000.0,NaN,NaN,NaN
0,1735671600,1735700070,1735725140,-12.33,-19.33,1030,85,-14.12,100,4.00,210,Clouds,overcast clouds,10000.0,NaN,NaN,NaN
0,1735675200,1735700070,1735725140,-14.00,-20.17,1030,89,-15.27,85,2.89,197,Clouds,overcast clouds,NaN,NaN,NaN,NaN
0,1735678800,1735700070,1735725140,-14.02,-20.01,1029,88,-15.41,84,2.76,191,Clouds,broken clouds,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1738324800,1738289955,1738320457,-17.37,-24.37,1030,77,-20.12,40,3.00,80,Snow,light snow,8000.0,NaN,NaN,NaN
0,1738328400,1738289955,1738320457,-17.82,-23.21,1030,78,-20.43,40,2.00,70,Snow,light snow,2600.0,NaN,NaN,NaN
0,1738332000,1738289955,1738320457,-17.93,-23.34,1030,79,-20.40,40,2.00,60,Snow,light snow,9000.0,NaN,NaN,NaN
0,1738335600,1738289955,1738320457,-17.63,-22.99,1030,79,-20.11,75,2.00,60,Snow,light snow,3900.0,NaN,NaN,NaN


In [57]:
excel_save_name_ts

'01-2025/01-2025_datestamps_1.'

In [81]:
feats_to_convert = ['dt', 'sunrise', 'sunset']

month_df_readable_date = month_df.copy()

def ts_convert(ts):
    return datetime.fromtimestamp(ts).strftime('%d-%m-%Y %H:%M:%S')

for feat in feats_to_convert:
    month_df_readable_date[feat] = month_df_readable_date[feat].apply(ts_convert)
    
    
readable_date_file_name = f'{saving_path}_readable_date.csv'

exist_files = os.listdir(f'{saving_path}')

n = 1

while readable_date_file_name in exist_files:
    print(f'in while, in {n} - iteration')
    readable_date_file_name_list = readable_date_file_name.split('.')
    
    if n == 1:
        readable_date_file_name = str(f'{readable_date_file_name_list[0]}_{n}.csv')
    else:
        orig_file_name = readable_date_file_name_list[0][:-2]
        readable_date_file_name = str(f'{orig_file_name}_{n}.csv')
            
    n += 1

month_df_readable_date.to_csv(f'{saving_path}/{readable_date_file_name}', index=False)

excel_save_name_readable_date = f'{saving_path}/{readable_date_file_name[:-3]}xlsx'

with pd.ExcelWriter(excel_save_name_readable_date, engine='xlsxwriter') as writer:  

    month_df_readable_date.to_excel(
        writer, sheet_name=f'{month}-{year}_hour_weather', index=False
        )
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(0, 2, 20)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(11, 11, 11)
    writer.sheets[f'{month}-{year}_hour_weather'].set_column(12, 12, 20)
    
    
    


In [5]:
jan_df_conv.to_csv('jan_df_gooddata.csv', index=False)

In [6]:
jan_df_conv

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,main,description,visibility,snow.1h,wind_gust,rain.1h
0,01-01-2025 00:00:00,01-01-2025 09:54:30,01-01-2025 16:52:20,-12.50,-19.50,1030,89,-13.78,100,3.87,208,Clouds,overcast clouds,NaN,NaN,NaN,NaN
0,01-01-2025 01:00:00,01-01-2025 09:54:30,01-01-2025 16:52:20,-12.26,-19.26,1030,92,-13.18,100,4.00,200,Snow,light snow,10000.0,NaN,NaN,NaN
0,01-01-2025 02:00:00,01-01-2025 09:54:30,01-01-2025 16:52:20,-12.33,-19.33,1030,85,-14.12,100,4.00,210,Clouds,overcast clouds,10000.0,NaN,NaN,NaN
0,01-01-2025 03:00:00,01-01-2025 09:54:30,01-01-2025 16:52:20,-14.00,-20.17,1030,89,-15.27,85,2.89,197,Clouds,overcast clouds,NaN,NaN,NaN,NaN
0,01-01-2025 04:00:00,01-01-2025 09:54:30,01-01-2025 16:52:20,-14.02,-20.01,1029,88,-15.41,84,2.76,191,Clouds,broken clouds,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,31-01-2025 19:00:00,31-01-2025 09:19:15,31-01-2025 17:47:37,-17.37,-24.37,1030,77,-20.12,40,3.00,80,Snow,light snow,8000.0,NaN,NaN,NaN
0,31-01-2025 20:00:00,31-01-2025 09:19:15,31-01-2025 17:47:37,-17.82,-23.21,1030,78,-20.43,40,2.00,70,Snow,light snow,2600.0,NaN,NaN,NaN
0,31-01-2025 21:00:00,31-01-2025 09:19:15,31-01-2025 17:47:37,-17.93,-23.34,1030,79,-20.40,40,2.00,60,Snow,light snow,9000.0,NaN,NaN,NaN
0,31-01-2025 22:00:00,31-01-2025 09:19:15,31-01-2025 17:47:37,-17.63,-22.99,1030,79,-20.11,75,2.00,60,Snow,light snow,3900.0,NaN,NaN,NaN
